In [1]:
!pip install transformers insightface huggingface onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 131.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.1 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp312-cp312-linux_x86_64.whl size=1071491 sha256=9bb30e0f971e044db7be552493ea497c8faa721198f88c78bfb8d4ce77088847
  Stored in directory: /root/.cache/pip/wheels/73/3c/e2/6d4815e8a8b33a2006554d65ce0d1f973e768f4c7a222fa675
Successfully built insightface


In [2]:
from huggingface_hub import snapshot_download
from insightface.app import FaceAnalysis
from insightface.model_zoo import get_model
import numpy as np
import cv2

In [3]:
snapshot_download(
    "fal/AuraFace-v1",
    local_dir="models/auraface",
)
face_app = FaceAnalysis(
    name="auraface",
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"],
    root="."
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

LICENSE.md: 0.00B [00:00, ?B/s]

glintr100.onnx:   0%|          | 0.00/261M [00:00<?, ?B/s]

...

genderage.onnx:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

1k3d68.onnx:   0%|          | 0.00/144M [00:00<?, ?B/s]

2d106det.onnx:   0%|          | 0.00/5.03M [00:00<?, ?B/s]

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: ./models/auraface/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecu

In [4]:
face_app.prepare(ctx_id=0)

set det-size: (640, 640)


In [23]:
image_paths = [
    "images1.jpg",
    "images2.jpg",
    "images3.jpg",
    "images4.jpg",
    "images5.png",
    "images6.png",
    "images7.png",
]

faces = []
embeddings = []

for p in image_paths:
    img = cv2.imread(p)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    bboxes, kpss = detector.detect(img, threshold=0.5)

    if bboxes is None:
        continue

    for bbox, kps in zip(bboxes, kpss):
        aligned = align_face(img, kps)

        faces.append(aligned)

        emb = face_app.models["recognition"].get_embedding(aligned)
        emb = emb / np.linalg.norm(emb)

        embeddings.append(emb)

AttributeError: 'ArcFaceONNX' object has no attribute 'get_embedding'

In [ ]:
embeddings = np.vstack(embeddings)
N = len(faces)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(embeddings)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(14, 14))
gs = GridSpec(
    nrows=N + 1,
    ncols=N + 1,
    width_ratios=[4] + [5]*N,
    height_ratios=[4] + [5]*N,
    wspace=0.05,
    hspace=0.05
)

# empty corner
ax = fig.add_subplot(gs[0, 0])
ax.axis("off")

# column labels
for j in range(N):
    ax = fig.add_subplot(gs[0, j + 1])
    ax.imshow(faces[j], aspect="auto")
    ax.axis("off")

# row labels
for i in range(N):
    ax = fig.add_subplot(gs[i + 1, 0])
    ax.imshow(faces[i], aspect="auto")
    ax.axis("off")

# similarity matrix
axm = fig.add_subplot(gs[1:, 1:])
im = axm.imshow(sim_matrix, cmap="magma", vmin=0.3, vmax=1.0)
axm.set_xticks([])
axm.set_yticks([])

for i in range(N):
    for j in range(N):
        axm.text(
            j, i,
            f"{sim_matrix[i, j]:.2f}",
            ha="center",
            va="center",
            color="white" if sim_matrix[i, j] < 0.7 else "black",
            fontsize=9
        )

cax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
plt.colorbar(im, cax=cax)
plt.suptitle("AuraFace Similarity Matrix", fontsize=16)
plt.show()


In [ ]:
image_paths = [
    "images5.png",
    "images6.png",
    "images7.png",
    "other1.png",
    "other2.png",
    "other3.png",
    "other4.png",
]

faces = []
embeddings = []

for p in image_paths:
    img = cv2.imread(p)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    detected = face_app.get(img)
    for f in detected:
        # SAFE face crop for visualization
        x1, y1, x2, y2 = map(int, f.bbox)
        crop = img[y1:y2, x1:x2]

        if crop.size == 0:
            continue

        faces.append(crop)
        embeddings.append(f.normed_embedding)  # already L2-normalized


In [ ]:
embeddings = np.vstack(embeddings)
N = len(faces)


In [ ]:

sim_matrix = cosine_similarity(embeddings)


In [ ]:

fig = plt.figure(figsize=(14, 14))
gs = GridSpec(
    nrows=N + 1,
    ncols=N + 1,
    width_ratios=[4] + [5]*N,
    height_ratios=[4] + [5]*N,
    wspace=0.05,
    hspace=0.05
)

# empty corner
ax = fig.add_subplot(gs[0, 0])
ax.axis("off")

# column labels
for j in range(N):
    ax = fig.add_subplot(gs[0, j + 1])
    ax.imshow(faces[j], aspect="auto")
    ax.axis("off")

# row labels
for i in range(N):
    ax = fig.add_subplot(gs[i + 1, 0])
    ax.imshow(faces[i], aspect="auto")
    ax.axis("off")

# similarity matrix
axm = fig.add_subplot(gs[1:, 1:])
im = axm.imshow(sim_matrix, cmap="magma", vmin=0.3, vmax=1.0)
axm.set_xticks([])
axm.set_yticks([])

for i in range(N):
    for j in range(N):
        axm.text(
            j, i,
            f"{sim_matrix[i, j]:.2f}",
            ha="center",
            va="center",
            color="white" if sim_matrix[i, j] < 0.7 else "black",
            fontsize=9
        )

cax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
plt.colorbar(im, cax=cax)
plt.suptitle("AuraFace Similarity Matrix", fontsize=16)
plt.show()